# 時中指數

# Load Data 

In [1]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./PTT_news_preprocessed.csv',sep='|')

In [2]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,2025-04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


In [3]:
df.shape

(268, 14)

# Filter news for selected keywords

In [4]:
import pandas as pd
from datetime import datetime, timedelta
import re

# # Load Data 
# df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [5]:
query_keywords = ['川普','Trump'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=16
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

In [6]:
len(df_query)

44

# Count how many pieces of news containing these keywords計算各類別多少篇文章提到該關鍵字

# Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [7]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['全部','學術股市','綜合','閒談','棒球','軍事']

def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0
        

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency


In [8]:
query_keywords = ['Trump','川普'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'全部': 207, '學術股市': 103, '綜合': 5, '閒談': 0, '棒球': 0, '軍事': 99},
 {'全部': 44, '學術股市': 22, '綜合': 3, '閒談': 0, '棒球': 0, '軍事': 19},
 44,
 207)

In [9]:
query_keywords = ['Trump','川普'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'全部': 207, '學術股市': 103, '綜合': 5, '閒談': 0, '棒球': 0, '軍事': 99},
 {'全部': 44, '學術股市': 22, '綜合': 3, '閒談': 0, '棒球': 0, '軍事': 19},
 44,
 207)

In [10]:
import re

'Trump','Donald Trump'

In [11]:
content="Trump'....，Donald Trump，Donald Trump，"
keyword="Trump"
re.findall(keyword, content)

['Trump', 'Trump', 'Trump']

In [12]:
len(re.findall(keyword, content))

3

In [13]:
keyword="Trump"
re.findall(keyword, content)

['Trump', 'Trump', 'Trump']

In [14]:
keyword="Donald Trump"
re.findall(keyword, content)

['Donald Trump', 'Donald Trump']

# Caclulate date-based reported frequency of these keywords計算被報導的次數以時間為基礎

In [15]:
def get_keyword_occurrence_time_series(df_query):
    date_samples = df_query.date
    query_freq = pd.DataFrame({'date_index': pd.to_datetime(date_samples), 'freq': [1 for _ in range(len(df_query))]})
    data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()
    line_xy_data = []
    for i, idx in enumerate(data.index):
        row = {'x': idx.strftime('%Y-%m-%d'), 'y': int(data.iloc[i].freq)}
        line_xy_data.append(row)
    return line_xy_data

In [16]:
get_keyword_occurrence_time_series(df_query)

[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]

# Put them together

In [17]:

query_keywords = ['Trump','川普'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'

df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
freqByDate = get_keyword_occurrence_time_series(df_query)


selectedCategories =  ['全部','學術股市','綜合','閒談','棒球','軍事']

# selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

freqByCate = [cate_occurrence[k] for k in selectedCategories]

response =  {'freqByDate': freqByDate, # 時間序列資料
           'freqByCate': freqByCate, # 各類別的篇數
           'category': selectedCategories, # 類別名稱
           'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
           'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
           }

In [18]:
freqByCate

[44, 22, 3, 0, 0, 19]

In [19]:
cate_freq

{'全部': 207, '學術股市': 103, '綜合': 5, '閒談': 0, '棒球': 0, '軍事': 99}

In [20]:
cate_occurrence

{'全部': 44, '學術股市': 22, '綜合': 3, '閒談': 0, '棒球': 0, '軍事': 19}

In [21]:
freqByDate

[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]

In [22]:
response

{'freqByDate': [{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}],
 'freqByCate': [44, 22, 3, 0, 0, 19],
 'category': ['全部', '學術股市', '綜合', '閒談', '棒球', '軍事'],
 'num_frequency': 207,
 'num_occurrence': 44}

In [23]:
response.items()

dict_items([('freqByDate', [{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]), ('freqByCate', [44, 22, 3, 0, 0, 19]), ('category', ['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']), ('num_frequency', 207), ('num_occurrence', 44)])

In [24]:
list(response.items())

[('freqByDate', [{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]),
 ('freqByCate', [44, 22, 3, 0, 0, 19]),
 ('category', ['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']),
 ('num_frequency', 207),
 ('num_occurrence', 44)]

## Save data to csv file

In [25]:
df_data = pd.DataFrame(list(response.items()),columns=['name','value'])

In [26]:
df_data

,name,value
0,freqByDate,"[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-..."
1,freqByCate,"[44, 22, 3, 0, 0, 19]"
2,category,"[全部, 學術股市, 綜合, 閒談, 棒球, 軍事]"
3,num_frequency,207
4,num_occurrence,44


In [27]:
## 存成csv格式檔案
df_data.to_csv('chen_shih_chung_data.csv',sep=',', index=None)

### Alternative way: using zip

In [28]:
k=list(response.keys())
v=list(response.values())

In [29]:
#list(zip(k,v))

In [30]:
df_data = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data

,name,value
0,freqByDate,"[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-..."
1,freqByCate,"[44, 22, 3, 0, 0, 19]"
2,category,"[全部, 學術股市, 綜合, 閒談, 棒球, 軍事]"
3,num_frequency,207
4,num_occurrence,44


## Read csv file and convert to dict format

In [31]:
df_data = pd.read_csv('chen_shih_chung_data.csv')

In [32]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-..."
1,freqByCate,"[44, 22, 3, 0, 0, 19]"
2,category,"['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']"
3,num_frequency,207
4,num_occurrence,44


In [33]:
# Convert to dictionary format
dict(list(df_data.values))

{'freqByDate': "[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]",
 'freqByCate': '[44, 22, 3, 0, 0, 19]',
 'category': "['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']",
 'num_frequency': '207',
 'num_occurrence': '44'}

In [34]:
list(df_data.values)

[array(['freqByDate',
        "[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]"],
       dtype=object),
 array(['freqByCate', '[44, 22, 3, 0, 0, 19]'], dtype=object),
 array(['category', "['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']"], dtype=object),
 array(['num_frequency', '207'], dtype=object),
 array(['num_occurrence', '44'], dtype=object)]

In [35]:
response = dict(list(df_data.values))


In [36]:
response['num_occurrence']

'44'

In [37]:
type(response['num_occurrence'])


str

In [38]:
type(response['freqByCate'])


str

In [39]:
response['freqByCate'] = eval(response['freqByCate'])


In [40]:
response


{'freqByDate': "[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-16', 'y': 34}]",
 'freqByCate': [44, 22, 3, 0, 0, 19],
 'category': "['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']",
 'num_frequency': '207',
 'num_occurrence': '44'}

### # How to convert a list into a dict?

In [41]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [42]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [43]:
dict([['one',[1,2,3]], ['two',2]])

{'one': [1, 2, 3], 'two': 2}

## All-in-one function

In [44]:
# Load Data 
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./PTT_news_preprocessed.csv',sep='|')

In [45]:


def process_data_all_in_one(query_keywords, weeks):


    cond='or'
    cate='全部'

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    cate_freq, cate_occurrence, _, _ = count_keyword(df_query, query_keywords)
    freqByDate = get_keyword_occurrence_time_series(df_query)


    selectedCategories = ['全部','學術股市','綜合','閒談','棒球','軍事']
    # selectedCategories = ['全部', '政治', '產經', '生活', '社會']
    # selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

    freqByCate = [cate_occurrence[k] for k in selectedCategories]

    response =  {'freqByDate': freqByDate,
            'freqByCate': freqByCate,
            'category': selectedCategories,
            'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
            'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
            }

    

    return response

In [46]:
query_keywords = ['Trump','川普'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [47]:
df_data = pd.DataFrame(list(data_response.items()),columns=['name','value'])
## 存成csv格式檔案
df_data.to_csv('Trump_data.csv',sep=',', index=None)

In [48]:
df_data = pd.read_csv('Trump_data.csv')

In [49]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-04-15', 'y': 10}, {'x': '2025-04-..."
1,freqByCate,"[44, 22, 3, 0, 0, 19]"
2,category,"['全部', '學術股市', '綜合', '閒談', '棒球', '軍事']"
3,num_frequency,207
4,num_occurrence,44


# This is your featured app

In [50]:
#query_keywords = ['蔡英文','小英'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
#query_keywords = ['賴清德'] #不要有子字串否則會重複計算 
#query_keywords = ['柯文哲','柯P','柯p'] #不要有子字串否則會重複計算 
query_keywords = ['蘋果','Apple','ipad','iphone','imac'] #不要有子字串否則會重複計算 
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [51]:
data_response

{'freqByDate': [{'x': '2025-04-16', 'y': 2}],
 'freqByCate': [2, 1, 1, 0, 0, 0],
 'category': ['全部', '學術股市', '綜合', '閒談', '棒球', '軍事'],
 'num_frequency': 2,
 'num_occurrence': 2}

In [52]:
query_keywords = ['疫情','新冠'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [53]:
data_response

{'freqByDate': [{'x': '2025-04-07', 'y': 1},
  {'x': '2025-04-08', 'y': 0},
  {'x': '2025-04-09', 'y': 0},
  {'x': '2025-04-10', 'y': 0},
  {'x': '2025-04-11', 'y': 0},
  {'x': '2025-04-12', 'y': 0},
  {'x': '2025-04-13', 'y': 0},
  {'x': '2025-04-14', 'y': 0},
  {'x': '2025-04-15', 'y': 0},
  {'x': '2025-04-16', 'y': 3}],
 'freqByCate': [4, 3, 0, 1, 0, 0],
 'category': ['全部', '學術股市', '綜合', '閒談', '棒球', '軍事'],
 'num_frequency': 5,
 'num_occurrence': 4}

# views.py in Django

In [54]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd

def load_data_scchen():
    # Read data from csv file
    df_data = pd.read_csv('./Trump_data.csv',sep=',')
    global response
    response = dict(list(df_data.values))
    del df_data

# load pk data
load_data_scchen()

def home(request):
    return render(request,'app_scchen/home.html', response)

print('app_scchen was loaded!')

app_scchen was loaded!


        # Saving data with other file format

        ## Save to text file
        # Save data
        f = open('pk_politician.txt','w')
        f.write(str(data_pk))
        f.close()

        # Load data
        f = open('pk_politician.txt','r')
        data_pk = f.read()

        # dictionary format
        eval(data_pk)
        ## Save to json file
        import json

        # Save data
        with open('pk_politician.txt', 'w') as file:
            json.dump( data_pk, file)

        # Load data
        with open('pk_politician.txt', 'r') as file:
            data_pk = json.load(file)

        # dictionary format
        data_pk

        ## save to mongoDb (A famous NoneSQL)
        from pymongo import MongoClient

        # Save data
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH = database["pk_politicianTsaiHan"]   # SQL: Table Name

        table_KTH.drop()
        table_KTH.insert_one(data)


        # Load data
        from pymongo import MongoClient
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH  = database["pkData"]   # SQL: Table Name

        for x in table_KTH.find():
            print(x)

        list(table_KTH.find({}, {'_id': False}))
